<a href="https://colab.research.google.com/github/Saira1302/Saira1302/blob/main/SNL_Summarization_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets evaluate rouge_score accelerate

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.2 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=3a6e563f3e17a14cd0ac84c0d0b5dca1af9b6f866db8ef77dbe15765c2a01835
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency 

In [2]:
import transformers
transformers.__version__

'4.44.2'

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
from huggingface_hub import list_datasets

datasets_list = list_datasets()

num = 0
for dataset in datasets_list:
  print(dataset)

  if (num > 10):
    break

  num += 1

DatasetInfo(id='amirveyseh/acronym_identification', author='amirveyseh', sha='15ef643450d589d5883e289ffadeb03563e80a9e', created_at=datetime.datetime(2022, 3, 2, 23, 29, 22, tzinfo=datetime.timezone.utc), last_modified=datetime.datetime(2024, 1, 9, 11, 39, 57, tzinfo=datetime.timezone.utc), private=False, gated=False, disabled=False, downloads=910, downloads_all_time=None, likes=19, paperswithcode_id='acronym-identification', tags=['task_categories:token-classification', 'annotations_creators:expert-generated', 'language_creators:found', 'multilinguality:monolingual', 'source_datasets:original', 'language:en', 'license:mit', 'size_categories:10K<n<100K', 'format:parquet', 'modality:text', 'library:datasets', 'library:pandas', 'library:mlcroissant', 'library:polars', 'arxiv:2010.14678', 'region:us', 'acronym-identification'], card_data=None, siblings=None)
DatasetInfo(id='ade-benchmark-corpus/ade_corpus_v2', author='ade-benchmark-corpus', sha='4ba01c71687dd7c996597042449448ea312126cf', 

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
from datasets import load_dataset

SNL_ds = load_dataset("navjordj/SNL_summarization")
SNL_ds

README.md:   0%|          | 0.00/2.97k [00:00<?, ?B/s]

(…)-00000-of-00001-7977b94df83881ae.parquet:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

(…)-00000-of-00001-97b50de59e7d677d.parquet:   0%|          | 0.00/1.18M [00:00<?, ?B/s]

(…)-00000-of-00001-476f99ddfb6075e4.parquet:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10874 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1300 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'date_scraped', 'headline', 'category', 'ingress', 'article'],
        num_rows: 10874
    })
    validation: Dataset({
        features: ['id', 'url', 'date_scraped', 'headline', 'category', 'ingress', 'article'],
        num_rows: 819
    })
    test: Dataset({
        features: ['id', 'url', 'date_scraped', 'headline', 'category', 'ingress', 'article'],
        num_rows: 1300
    })
})

In [6]:
SNL_ds = load_dataset("navjordj/SNL_summarization", split="train[:3%]")
SNL_ds

Dataset({
    features: ['id', 'url', 'date_scraped', 'headline', 'category', 'ingress', 'article'],
    num_rows: 326
})

In [7]:
SNL_ds=SNL_ds.train_test_split(test_size=0.2, seed=42)
SNL_ds

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'date_scraped', 'headline', 'category', 'ingress', 'article'],
        num_rows: 260
    })
    test: Dataset({
        features: ['id', 'url', 'date_scraped', 'headline', 'category', 'ingress', 'article'],
        num_rows: 66
    })
})

In [8]:
def clean_txt(example):
    for txt in ["category", "article"]:
       example[txt]  = example[txt].lower()
       example[txt]  = example[txt].replace("/", "")
       example[txt]  = example[txt].replace("\n", "")
       example[txt]  = example[txt].replace("``", "")
       example[txt]  = example[txt].replace('"', '')
       example[txt]  = example[txt].replace("--", "")

    return example

In [9]:
cleaned_SNL_ds = SNL_ds.map(clean_txt)
cleaned_SNL_ds

Map:   0%|          | 0/260 [00:00<?, ? examples/s]

Map:   0%|          | 0/66 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'date_scraped', 'headline', 'category', 'ingress', 'article'],
        num_rows: 260
    })
    test: Dataset({
        features: ['id', 'url', 'date_scraped', 'headline', 'category', 'ingress', 'article'],
        num_rows: 66
    })
})

In [10]:
cleaned_SNL_ds['train']["article"][7]

"albertus magnus studerte i bologna og padova, hvor han i 1222 trådte inn i dominikanerordenen. fra 1245 til 1248 underviste han ved universitetet i paris med blant andre thomas aquinas som student. aquinas fulgte sin lærer til köln da denne i 1248 ble sendt dit for å opprette et dominikansk studiekloster. i 1260 ble albertus biskop av regensburg, men nedla embetet allerede to år senere. resten av livet bodde han for det meste i köln. han deltok i kirkemøtet i lyon 1274.albertus magnus var en av de første kristne teologene som kommenterte aristoteles' filosofi og søkte å integrere den i kristen tenkning. hans verk ble i så måte videreført av eleven thomas aquinas. begge var innstilt på å løse filosofiske problemer, ikke som teologer, men etter filosofiens egne metoder. man kan også finne innflytelse fra augustin og nyplatonismen i hans verker. han var opptatt av de arabiske filosofene, særlig averroës (ibn rusjd), og hans interesser strakte seg dessuten til naturvitenskapelige observas

In [11]:
 SNL_ds['train']["article"][0]

'På et seilskip danner baugsprydet nedre feste til forseilene klyver, jager og stengestagseil med deres stag. Baugsprydets rigging trekker det ned mot forstevn og vannflaten, ettersom stagene opp til den øvrige riggen trekker baugsprydet i oppadgående retning. Den nedre riggingen av baugsprydet består av et vaterstag, en martingal, og ofte sidestag, som stiver av mot horisontale krefter i sprydet.Baugspryd har også blitt vanlig på moderne seilbåter, både kjølbåter og joller, som feste for fremre hjørne av såkalte genakkerseil og til dels spinnakere. Større seilfartøy i moderne tid fører også normalt et baugspryd, dersom det er hensiktsmessig for riggingen av skipet. Fordelene med baugsprydene er at en kan bygge en noe større rigg, uten å måtte forlenge selve skroget på skipet, som i sin tur ville krevd en enda større rigg. Ved å flytte spesielt de forre stagseilene lengre fram vil skipet ofte dra fordel av bedre eller mer allsidige seilegenskaper.Seilskip ble ofte bygget med en gallion

In [12]:
 cleaned_SNL_ds['train']["article"][0]

'på et seilskip danner baugsprydet nedre feste til forseilene klyver, jager og stengestagseil med deres stag. baugsprydets rigging trekker det ned mot forstevn og vannflaten, ettersom stagene opp til den øvrige riggen trekker baugsprydet i oppadgående retning. den nedre riggingen av baugsprydet består av et vaterstag, en martingal, og ofte sidestag, som stiver av mot horisontale krefter i sprydet.baugspryd har også blitt vanlig på moderne seilbåter, både kjølbåter og joller, som feste for fremre hjørne av såkalte genakkerseil og til dels spinnakere. større seilfartøy i moderne tid fører også normalt et baugspryd, dersom det er hensiktsmessig for riggingen av skipet. fordelene med baugsprydene er at en kan bygge en noe større rigg, uten å måtte forlenge selve skroget på skipet, som i sin tur ville krevd en enda større rigg. ved å flytte spesielt de forre stagseilene lengre fram vil skipet ofte dra fordel av bedre eller mer allsidige seilegenskaper.seilskip ble ofte bygget med en gallion

In [13]:
EXAMPLE_TEXT_INDEX = 0
example_text = cleaned_SNL_ds["test"]["article"][EXAMPLE_TEXT_INDEX]
example_text

"bakgrunn dickens blei fødd i portsmouth, men voks opp i london under vanskelege forhold. mr. micawber i david copperfield er eit satirisk portrett av faren, som blei send i fengsel på grunn av gjeld då charles var berre 12 år gammal. guten blei sett i arbeid i ein svertefabrikk, ei audmjukande oppleving som sette sterkt preg på han. skulegangen blei uregelmessig, så han las mykje på eiga hand og studerte folkelivet i londons gater. i 1827 blei han tilsett som lærling på eit sakførarkontor, samstundes som han lærte seg stenografi for å tene pengar som rettsreferent og parlamentsstenograf. journalistiske skisser då han i 1833 byrja å offentleggjere skissene sine frå london (seinare gitt ut som sketches by boz), blei det klart at han var litt meir enn ein vanleg journalist. med the posthumous papers of the pickwick club (1836–1837) var suksessen sikra, og han var snar til å utnytte han. populære hefte det kan trygt seiast at det var dickens som populariserte den engelske romanen. han rev

In [14]:
MODEL_NAME= "navjordj/snl-summarization"

In [15]:
from transformers import pipeline

prefix = "summarize: "

summarizer = pipeline("summarization", model = MODEL_NAME)

summary_txt = summarizer(prefix + example_text)

summary_txt

config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/489 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'summary_text': 'charles dickens er ein engelsk forfattar. Han skreiv ei rekkje romanar,'}]

In [16]:
summarizer = pipeline("summarization", model = MODEL_NAME, truncation=True)
summary_txt = summarizer(prefix + example_text)
summary_txt

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'summary_text': 'charles dickens er ein engelsk forfattar. Han skreiv ei rekkje romanar,'}]

In [17]:
ref_txt = cleaned_SNL_ds["test"]["category"][EXAMPLE_TEXT_INDEX]
ref_txt

'store norske leksikon,språk og litteratur,litteratur i verden,europa,storbritannia og nord-irlands litteratur,storbritannia og nord-irlands litteratur på 1800-tallet'

In [18]:
import evaluate
rouge = evaluate.load("rouge")
rouge

EvaluationModule(name: "rouge", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id=None)}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLsum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/

In [19]:
reference = "Albrechts karriere ble definert av hans store engasjement for sjeldent fremført repertoar, noe som ga ham et rykte for å gjenopplive skjulte musikalske perler."

ex_1 = "Albrechts dedikasjon til å avdekke skjulte musikalske perler var tydelig i hans store engasjement for sjeldent fremført repertoar, og brakte glemte stykker til live på scenen."

ex_2 = "Hans lidenskap for mangfold i klassisk musikkprogrammering førte til at Albrecht fokuserte på sjeldent fremført repertoar, og ga publikum en unik lytteopplevelse."


In [20]:
results = rouge.compute(predictions = [ex_1], references = [reference])
results

{'rouge1': 0.48000000000000004,
 'rouge2': 0.37499999999999994,
 'rougeL': 0.35999999999999993,
 'rougeLsum': 0.35999999999999993}

In [21]:
results = rouge.compute(predictions = [ex_2], references = [reference])
results

{'rouge1': 0.2916666666666667,
 'rouge2': 0.13043478260869565,
 'rougeL': 0.2916666666666667,
 'rougeLsum': 0.2916666666666667}

In [22]:
summary_txt[0]["summary_text"]

'charles dickens er ein engelsk forfattar. Han skreiv ei rekkje romanar,'

In [23]:
result = rouge.compute(predictions = [summary_txt[0]["summary_text"]],
                       references = [ref_txt], use_stemmer = True)
result

{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'rougeLsum': 0.0}

In [24]:
article_texts = cleaned_SNL_ds["test"]["article"]

article_categories = cleaned_SNL_ds["test"]["category"]

In [26]:
from tqdm import tqdm

candidate_summaries = []

prefix = "summarize: "

for i, text in enumerate(tqdm(article_texts[:50])):

    candidate = summarizer(prefix + text)

    candidate_summaries.append(candidate[0]["summary_text"])

100%|██████████| 50/50 [03:53<00:00,  4.67s/it]


In [27]:
result_agg = rouge.compute(predictions = candidate_summaries,
                           references = article_categories[:50],
                           use_stemmer = True)
result_agg

{'rouge1': 0.059774973859326425,
 'rouge2': 0.004926686217008797,
 'rougeL': 0.056330679506336426,
 'rougeLsum': 0.05723583943024486}

In [28]:
result_unagg = rouge.compute(predictions = candidate_summaries,
                             references = article_categories[:50],
                             use_stemmer = True,
                             use_aggregator = False)
result_unagg

{'rouge1': [0.0,
  0.0,
  0.09523809523809525,
  0.0,
  0.10526315789473684,
  0.1904761904761905,
  0.14285714285714288,
  0.0,
  0.33333333333333337,
  0.17391304347826086,
  0.08695652173913043,
  0.1904761904761905,
  0.0,
  0.09523809523809525,
  0.0,
  0.07142857142857144,
  0.0,
  0.10526315789473685,
  0.13793103448275862,
  0.0,
  0.0,
  0.10000000000000002,
  0.0,
  0.0,
  0.0,
  0.08695652173913043,
  0.0,
  0.0,
  0.0,
  0.10526315789473685,
  0.1111111111111111,
  0.1818181818181818,
  0.0,
  0.0,
  0.08,
  0.08695652173913043,
  0.08333333333333333,
  0.07692307692307691,
  0.0,
  0.0,
  0.08333333333333333,
  0.052631578947368425,
  0.0,
  0.08695652173913043,
  0.0,
  0.07142857142857144,
  0.0,
  0.07407407407407407,
  0.0,
  0.0],
 'rouge2': [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.1818181818181818,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,

In [29]:
import numpy as np

result_unagg_rsum = np.array(result_unagg["rouge2"])

np.argmax(result_unagg_rsum), np.argmin(result_unagg_rsum)

(8, 0)

In [30]:
import pandas as pd

act_vs_pred_summaries_df = pd.DataFrame(list(zip(candidate_summaries, article_categories[:50])),
                                        columns = ["Predicted_Summaries", "Reference_summaries"])
act_vs_pred_summaries_df.head()

,Predicted_Summaries,Reference_summaries
0,charles dickens er ein engelsk forfattar. Han ...,"store norske leksikon,språk og litteratur,litt..."
1,François césar var en fransk maler. Han var en...,"store norske leksikon,kunst og estetikk,bilden..."
2,William bennett jr. var en britisk journalist ...,"store norske leksikon,samfunn,medier,aviser,av..."
3,Andrzej dmowski var en polsk politiker. Han va...,"store norske leksikon,historie,verdens histori..."
4,William broom var en australsk paleontolog. Ha...,"store norske leksikon,realfag,paleontologi,pal..."


In [31]:
print("Actual Summary")
print(act_vs_pred_summaries_df._get_value(20, "Predicted_Summaries"))

print("Reference Summary")
print(act_vs_pred_summaries_df._get_value(20, "Reference_summaries"))

Actual Summary
Brønnoperatør er en petroleumsoperatør som arbeider i petroleumsvirksomhet.
Reference Summary
store norske leksikon,samfunn,utdanning og forskning,grunnopplæring,yrkesfaglig utdanning,teknologi- og industrifag


In [32]:
print("Actual Summary")
print(act_vs_pred_summaries_df._get_value(18, "Predicted_Summaries"))

print("Reference Summary")
print(act_vs_pred_summaries_df._get_value(18, "Reference_summaries"))

Actual Summary
Bruce alden er en amerikansk jazzmusiker, komponist og komponist. Han er
Reference Summary
store norske leksikon,kunst og estetikk,musikk,jazz,nord-amerikanske jazzutøvere og -ensembler, 1980 til i dag


In [33]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [34]:
tokenizer(["four-time defending champion"])

{'input_ids': [[9171, 264, 3315, 44446, 347, 69624, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1]]}

In [35]:
tokens = tokenizer.tokenize("four-time defending champion")
tokens


['▁four', '-', 'time', '▁defend', 'ing', '▁champion']

In [36]:
ids = tokenizer.convert_tokens_to_ids(tokens)
ids

[9171, 264, 3315, 44446, 347, 69624]

In [37]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

In [38]:
prefix = "summarize: "


def preprocess_function(examples):

    inputs = [prefix + doc for doc in examples["category"]]
    model_inputs = tokenizer(inputs, max_length = 512, truncation = True)

    labels = tokenizer(text_target = examples["article"], max_length = 128, truncation = True)

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [39]:
tokenized_SNL_ds = cleaned_SNL_ds.map(preprocess_function, batched = True)

tokenized_SNL_ds

Map:   0%|          | 0/260 [00:00<?, ? examples/s]

Map:   0%|          | 0/66 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'date_scraped', 'headline', 'category', 'ingress', 'article', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 260
    })
    test: Dataset({
        features: ['id', 'url', 'date_scraped', 'headline', 'category', 'ingress', 'article', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 66
    })
})

In [40]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer, model = MODEL_NAME)

In [41]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens = True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens = True)

    result = rouge.compute(predictions = decoded_preds, references = decoded_labels, use_stemmer = True)

    prediction_length = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["generated_length"] = np.mean(prediction_length)

    return {k: round(v, 4) for k, v in result.items()}

In [42]:
from huggingface_hub import notebook_login
notebook_login()

In [43]:
training_args = Seq2SeqTrainingArguments(
    output_dir = "Fine-Tuned-SNL-Summarization",
    evaluation_strategy = "epoch",
    learning_rate = 2e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    weight_decay = 0.01,
    save_total_limit = 3,
    num_train_epochs = 3,
    predict_with_generate = True,
    fp16 = True,
    push_to_hub = True,
)

trainer = Seq2SeqTrainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_SNL_ds["train"],
    eval_dataset = tokenized_SNL_ds["test"],
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Generated Length
1,No log,nan,0.070200,0.004600,0.055400,0.055900,19.000000
2,No log,nan,0.070200,0.004600,0.055400,0.055900,19.000000
3,No log,nan,0.070200,0.004600,0.055400,0.055900,19.000000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=51, training_loss=0.0, metrics={'train_runtime': 96.0958, 'train_samples_per_second': 8.117, 'train_steps_per_second': 0.531, 'total_flos': 81076127109120.0, 'train_loss': 0.0, 'epoch': 3.0})

In [44]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/Themarina/Fine-Tuned-SNL-Summarization/commit/a806a52e42b8df053a5e4e8dfb6f8aec69df5c72', commit_message='End of training', commit_description='', oid='a806a52e42b8df053a5e4e8dfb6f8aec69df5c72', pr_url=None, pr_revision=None, pr_num=None)